In [262]:
import sympy

# 3変数に対する解法
n = 3
x, y, z = sympy.symbols('x y z')

def grad(f):
    return sympy.Matrix([[sympy.diff(f, x)], [sympy.diff(f, y)], [sympy.diff(f, z)]])


def solve(X, k):
    # データ点の数と次元を取得する
    m, d = X.shape
    print(f"データ点の数: {m}, 次元: {d}")

    # Veronese Mapの出力を定義する
    V_terms = sympy.expand((x + y + z) ** k)
    V_out = sympy.Matrix([[term for term, _ in V_terms.as_coefficients_dict().items()]])
    print(f"Veronese Out: {V_out}")

    # Veronese Mapを計算する
    V = sympy.Matrix([V_out.subs({x: X.row(m_i)[0], y: X.row(m_i)[1], z: X.row(m_i)[2]}) for m_i in range(m)])
    V_rank = V.rank()
    print(f"Veronese Map [rank {V_rank}]: {V}")

    # rを計算する
    M_k = sympy.functions.combinatorial.numbers.nC(n + k - 1, n - 1)
    r = M_k - V_rank
    print(f"r: {r}")

    # Veronese Mapの固有値0の固有ベクトルを計算
    VtA = V.T * V
    eigenvalues_vectors = VtA.eigenvects()
    zero_eigenvalues_vectors = list(filter(lambda x: x[0] == 0, eigenvalues_vectors))
    if len(zero_eigenvalues_vectors) == 0:
        raise ValueError("固有値0の固有ベクトルが見つかりません")

    zero_eigenvectors = zero_eigenvalues_vectors[0][2]
    
    # grad fを計算する
    f_grads = sympy.Matrix([[]])
    for zero_eigenvector in zero_eigenvectors:
        # fを求める
        f = V_out.dot(zero_eigenvector)

        # fの勾配を計算する
        f_grad = grad(f)
        f_grads = sympy.Matrix([[f_grads, f_grad]])

    print(f"grad f: {f_grads}")
    return f_grads


# example 1

In [274]:
# データ点
X = sympy.Matrix([
    [0, 0, 1],
    [0, 0, -1],
    [1, 1, 0],
    [1, -1, 0],
    [0, -1, 0],
    [-1, 1, 0]
])

# クラスタの数
k = 2

# サンプル点
sample_points = [{x: 0, y: 0, z: 1}, {x: 1, y: 1, z: 0}]
print(sample_points)

# 解く
try: 
    f_grads = solve(X, k)
except:  
    import traceback
    traceback.print_exc()
    assert False

print("Answer ------------------------------------------------------")
for i, sample_point in enumerate(sample_points):
    print(f"[S_{i}]")
    S_i = f_grads.subs(sample_point)
    print(f"codimention: {S_i.rank()}")
    print(f"basis: {[column / column.norm() for column in S_i.columnspace()]}")
print("-------------------------------------------------------------")

[{x: 0, y: 0, z: 1}, {x: 1, y: 1, z: 0}]
データ点の数: 6, 次元: 3
Veronese Out: Matrix([[x**2, y**2, z**2, x*y, x*z, y*z]])
Veronese Map [rank 4]: Matrix([[0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0], [1, 1, 0, -1, 0, 0], [0, 1, 0, 0, 0, 0], [1, 1, 0, -1, 0, 0]])
r: 2
grad f: Matrix([[z, 0], [0, z], [x, y]])
Answer ------------------------------------------------------
[S_0]
codimention: 2
basis: [Matrix([
[1],
[0],
[0]]), Matrix([
[0],
[1],
[0]])]
[S_1]
codimention: 1
basis: [Matrix([
[0],
[0],
[1]])]
-------------------------------------------------------------


# example 2

In [273]:
# データ点
X = sympy.Matrix([
    [0, 1, 1],
    [1, 0, -1],
    [1, 1, 1],
    [1, -1, 1],
    [0, -1, 1],
    [0, 0, 1]
])

# クラスタの数
k = 3

# サンプル点
sample_points = [{x: X.row(X_i)[0], y: X.row(X_i)[1], z: X.row(X_i)[2]} for X_i in range(X.rows)]

# 解く
try: 
    f_grads = solve(X, k)
except:  
    import traceback
    traceback.print_exc()
    assert False

print("Answer ------------------------------------------------------")
for i, sample_point in enumerate(sample_points):
    print(f"[S_{i}]")
    S_i = f_grads.subs(sample_point)
    print(f"codimention: {S_i.rank()}")
    print(f"basis: {[column / column.norm() for column in S_i.columnspace()]}")
print("-------------------------------------------------------------")

データ点の数: 6, 次元: 3
Veronese Out: Matrix([[x**3, y**3, z**3, x*y**2, x*z**2, x**2*y, y*z**2, x**2*z, y**2*z, x*y*z]])
Veronese Map [rank 6]: Matrix([[0, 1, 1, 0, 0, 0, 1, 0, 1, 0], [1, 0, -1, 0, 1, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, -1, 1, 1, 1, -1, -1, 1, 1, -1], [0, -1, 1, 0, 0, 0, -1, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])
r: 4
grad f: Matrix([[-3*x**2 + z**2, 0, 3*x**2 + 2*x*z - 2*y**2, -2*x*y + y*z], [0, -3*y**2 + z**2, -4*x*y, -x**2 + x*z], [2*x*z, 2*y*z, x**2, x*y]])
Answer ------------------------------------------------------
[S_0]
codimention: 2
basis: [Matrix([
[1],
[0],
[0]]), Matrix([
[         0],
[-sqrt(2)/2],
[ sqrt(2)/2]])]
[S_1]
codimention: 2
basis: [Matrix([
[-sqrt(2)/2],
[         0],
[-sqrt(2)/2]]), Matrix([
[0],
[1],
[0]])]
[S_2]
codimention: 2
basis: [Matrix([
[-sqrt(2)/2],
[         0],
[ sqrt(2)/2]]), Matrix([
[         0],
[-sqrt(2)/2],
[ sqrt(2)/2]])]
[S_3]
codimention: 2
basis: [Matrix([
[-sqrt(2)/2],
[         0],
[ sqrt(2)/2]]), Matrix(